In [2]:
from ase import Atoms
from ase.data import atomic_numbers, covalent_radii
from ase import neighborlist
from clus_utils import  addAtoms, fixOverlap, checkBonded, checkSimilar
from ase.calculators.emt import EMT
import numpy as np
from ase.optimize import BFGS
import random
from ase.io  import read, write, Trajectory

In [3]:
def ase_to_list(clus):
        list_coord = []
        for i in range(len(clus)):
                elem, x, y, z = clus.get_chemical_symbols()[i],  clus.get_positions()[i][0], clus.get_positions()[i][1], clus.get_positions()[i][2]
                list_coord.append([elem, x, y , z])    
        clus = list_coord
        return clus

In [4]:
#

In [5]:
DIRECTION =[np.array([1,0,0]),
           np.array([-1,0,0]),
           np.array([0,1,0]),
           np.array([0,-1,0]),
           np.array([0,0,1]),
           np.array([0,0,-1]),
          ]

class ClusEnv():
    
    def __init__(self,
                 eleNames=None,
                 eleNums=None
                ):
        
        self.eleNames = eleNames
        self.eleNums= eleNums
        self.eleRadii = [covalent_radii[atomic_numbers[ele]] for ele in eleNames]
        self.seed = 100
    
    def gen_clus(self):
        ele_initial = [self.eleNames[0], self.eleNames[-1]]
        d = (self.eleRadii[0] + self.eleRadii[-1]) / 2
        clusm = Atoms(ele_initial, [(-d, 0.0, 0.0), (d, 0.0, 0.0)])
        clus = addAtoms(clusm, self.eleNames, self.eleNums, self.eleRadii, self.seed)
        clus = fixOverlap(clus)
        return clus, d

In [6]:
import random
shift_move = random.choice(DIRECTION)
print(shift_move)

[ 0 -1  0]


In [7]:
ele_Names = ['Ni']
ele_Nums = [8]
clus_example = ClusEnv(ele_Names, ele_Nums)

In [8]:
clus_example.eleNums

[8]

In [9]:
clus_ex1,d = clus_example.gen_clus()
print(clus_ex1)
print(d)

Atoms(symbols='Ni8', pbc=True, cell=[22.476408257419912, 24.757224523586167, 21.445956314611653], calculator=EMT(...))
1.24


In [10]:
print(clus_ex1.get_positions())
print(clus_ex1.get_potential_energy())

[[ 9.14795472 13.29311352 11.21427005]
 [11.39683897 12.62050761 11.97393969]
 [10.72578025 14.9645181  11.96131213]
 [10.3375256  11.55059875  9.99042448]
 [ 9.49096169 11.12674832 12.2275232 ]
 [11.60442279 10.2688204  11.54775543]
 [12.92006015 14.34479348 11.28625192]
 [11.09164477 13.84491073  9.83883306]]
11.24761791524499


In [11]:
clus_ex1.get_scaled_positions()

array([[0.40700252, 0.53693876, 0.52290837],
       [0.50705784, 0.5097707 , 0.55833088],
       [0.4772017 , 0.60445055, 0.55774207],
       [0.45992783, 0.46655467, 0.46584187],
       [0.42226327, 0.4494344 , 0.57015519],
       [0.51629347, 0.41478076, 0.53845841],
       [0.57482761, 0.57941848, 0.5262648 ],
       [0.49347941, 0.55922709, 0.45877334]])

In [12]:
elements = np.array(clus_ex1.symbols)
elements

array(['Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni'], dtype='<U2')

In [13]:
 _, idx = np.unique(elements, return_index=True)
        

In [14]:
np.unique(elements, return_index=True)

(array(['Ni'], dtype='<U2'), array([0]))

In [15]:
elements = list(elements[np.sort(idx)])
elements

['Ni']

In [16]:
print(clus_ex1.get_positions())
print(clus_ex1.get_potential_energy())

[[ 9.14795472 13.29311352 11.21427005]
 [11.39683897 12.62050761 11.97393969]
 [10.72578025 14.9645181  11.96131213]
 [10.3375256  11.55059875  9.99042448]
 [ 9.49096169 11.12674832 12.2275232 ]
 [11.60442279 10.2688204  11.54775543]
 [12.92006015 14.34479348 11.28625192]
 [11.09164477 13.84491073  9.83883306]]
11.24761791524499


In [17]:
clus_ex1_list = ase_to_list(clus_ex1)
clus_ex1_list
for item in clus_ex1_list:
  print(item[0], item[1], item[2], item[3])

Ni 9.147954723017435 13.293113519981032 11.214270045386668
Ni 11.39683896828267 12.620507612572993 11.973939687982954
Ni 10.725780248982089 14.964518096429165 11.961312127540511
Ni 10.337525598725897 11.550598753539486 9.990424484525994
Ni 9.490961687642251 11.126748316424575 12.227523195645846
Ni 11.604422791798804 10.268820402061195 11.54775543210759
Ni 12.92006015199387 14.34479347844378 11.286251923241972
Ni 11.091644768726722 13.84491072541212 9.838833064921147


In [20]:
i = 7
dist_before = clus_ex1.get_all_distances()[i]
print(dist_before)

[2.44422545 2.4801183  2.42740497 2.41982295 3.95682117 3.99647081
 2.38495428 0.        ]


In [21]:
print(clus_ex1.get_positions()[i])
b = clus_ex1.get_positions()[i] + shift_move
print(b)

[11.09164477 13.84491073  9.83883306]
[11.09164477 12.84491073  9.83883306]


In [22]:
clus_ex2 = clus_ex1.copy()
clus_ex2[i].position = b
clus_ex2.calc = EMT()
clus_ex2.get_positions()[i]

array([11.09164477, 12.84491073,  9.83883306])

In [23]:
print(clus_ex2.get_positions())
print(clus_ex2.get_potential_energy())

[[ 9.14795472 13.29311352 11.21427005]
 [11.39683897 12.62050761 11.97393969]
 [10.72578025 14.9645181  11.96131213]
 [10.3375256  11.55059875  9.99042448]
 [ 9.49096169 11.12674832 12.2275232 ]
 [11.60442279 10.2688204  11.54775543]
 [12.92006015 14.34479348 11.28625192]
 [11.09164477 12.84491073  9.83883306]]
24.11180107592081


In [24]:
clus_ex2_list = ase_to_list(clus_ex2)
clus_ex2_list
for item in clus_ex2_list:
  print(item[0], item[1], item[2], item[3])

Ni 9.147954723017435 13.293113519981032 11.214270045386668
Ni 11.39683896828267 12.620507612572993 11.973939687982954
Ni 10.725780248982089 14.964518096429165 11.961312127540511
Ni 10.337525598725897 11.550598753539486 9.990424484525994
Ni 9.490961687642251 11.126748316424575 12.227523195645846
Ni 11.604422791798804 10.268820402061195 11.54775543210759
Ni 12.92006015199387 14.34479347844378 11.286251923241972
Ni 11.091644768726722 12.84491072541212 9.838833064921147


In [25]:
dist_after = clus_ex2.get_all_distances()[i]
print(dist_after)

[2.42294111 2.16845119 3.02183878 1.50562916 3.3496431  3.13362383
 2.77268326 0.        ]


In [26]:
z = []
for k, dist in enumerate(dist_after):
   
    print(dist)
    if  k != i and dist < d:
        z.append(dist)
print(z)

z1 = [ dist for k, dist in enumerate(dist_after) if  k != i and dist < d ]
print(z1)

2.422941110733165
2.168451186523893
3.0218387821548873
1.505629158057645
3.3496431049217077
3.133623830659657
2.772683255987337
0.0
[]
[]


In [27]:
dyn= BFGS(clus_ex2)
dyn.run(fmax=0.02, steps=1000)


      Step     Time          Energy         fmax
BFGS:    0 20:33:29       24.111801       53.8284
BFGS:    1 20:33:29       13.313217       10.8212
BFGS:    2 20:33:29       12.459739        6.7661
BFGS:    3 20:33:29       11.736747        2.7096
BFGS:    4 20:33:30       11.539667        1.3349
BFGS:    5 20:33:30       11.456270        0.7272
BFGS:    6 20:33:30       11.420481        0.5462
BFGS:    7 20:33:30       11.397648        0.4933
BFGS:    8 20:33:30       11.377532        0.4304
BFGS:    9 20:33:30       11.359318        0.3360
BFGS:   10 20:33:30       11.346265        0.3939
BFGS:   11 20:33:30       11.336204        0.3535
BFGS:   12 20:33:30       11.326115        0.3021
BFGS:   13 20:33:30       11.313937        0.2643
BFGS:   14 20:33:30       11.305551        0.2731
BFGS:   15 20:33:30       11.298886        0.2363
BFGS:   16 20:33:30       11.293071        0.1981
BFGS:   17 20:33:30       11.288300        0.1702
BFGS:   18 20:33:30       11.282782        0.2891
B

True

In [29]:
clus_ex2.get_potential_energy()

11.2479363627137

In [31]:
clus_ex2.get_distances(i, [0,1,2,3,4,5,6,7])

array([2.44178906, 2.49223393, 2.42272808, 2.41747379, 3.96096193,
       4.00284827, 2.38275551, 0.        ])

In [32]:
bb = checkBonded(clus_ex2)
bb

True

In [33]:
print(checkSimilar(clus_ex2, clus_ex1))

True


In [34]:
print(clus_ex1.get_moments_of_inertia())
print(clus_ex2.get_moments_of_inertia())

[ 867.42832006 1519.72581766 1730.20784101]
[ 867.5385007  1515.75543306 1728.8148248 ]


In [35]:
print(clus_ex2.get_potential_energy())
print(clus_ex1.get_potential_energy())

11.2479363627137
11.24761791524499


In [36]:
min_ene_list = []
def test():
    ele_Names = ['Pt']
    ele_Nums = [20]
    clus_example = ClusEnv(ele_Names, ele_Nums)
    clus_ex1,d = clus_example.gen_clus()
    
    random.seed()
    i = random.randint(0,len(clus_ex1)-1)
    shift_move = random.choice(DIRECTION)
    #print(i,shift_move)

    clus_ex2 = clus_ex1.copy()
    clus_ex2[i].position =  clus_ex1.get_positions()[i] + shift_move * 2.5
    
    dist_after = clus_ex2.get_all_distances()[i]
    z1 = [ dist for k, dist in enumerate(dist_after) if  k != i and dist < d ]
    
    clus_ex2.calc = EMT()
    dyn= BFGS(clus_ex2, logfile='test.log')
    dyn.run(fmax=0.02, steps=1000)

    ene1 = clus_ex1.get_potential_energy()
    ene2 = clus_ex2.get_potential_energy()
    diff_ene = ene2 - ene1

    diff_iner = clus_ex2.get_moments_of_inertia() - clus_ex1.get_moments_of_inertia()
    
    if  checkSimilar(clus_ex2, clus_ex1):
    #xx checkSimilar(clus_ex2, clus_ex1)
        print(i,shift_move, len(z1), checkSimilar(clus_ex2, clus_ex1), ene2, ene1, diff_iner )
        min_ene_list.append(ene2)
    return 

In [ ]:
'''
for i in range(250):
    test()
print(min_ene_list)
print(len(min_ene_list))
print(sorted(min_ene_list))
'''

In [37]:
min_ene_list = []
minima_list = []
traj_atom_list = []
reward_list  = []
def test_new():
    ele_Names = ['Pd']
    ele_Nums = [13]
    clus_example = ClusEnv(ele_Names, ele_Nums)
    clus_ex1,d = clus_example.gen_clus()
    
    ene1 = clus_ex1.get_potential_energy()
    if clus_ex1 not in minima_list:
        minima_list.append(clus_ex1)
        traj_atom_list.append(clus_ex1)
    if ene1 not in min_ene_list:
        min_ene_list.append(ene1)
    
    
    random.seed()
    i = random.randint(0,len(clus_ex1)-1)
    shift_move = random.choice(DIRECTION)
    #print(i,shift_move)

    clus_ex2 = clus_ex1.copy()
    clus_ex2[i].position =  clus_ex1.get_positions()[i] + shift_move * 2.5
    
    dist_after = clus_ex2.get_all_distances()[i]
    z1 = [ dist for k, dist in enumerate(dist_after) if  k != i and dist < d ]
    
    #reward = 0.0
    if len(z1) > 0:
        reward = 1000.0
        reward_list.append(reward)
    
        
    clus_ex2.calc = EMT()
    dyn= BFGS(clus_ex2, logfile='test.log')
    dyn.run(fmax=0.02, steps=1000)

    
    ene2 = clus_ex2.get_potential_energy()
    diff_ene = ene2 - ene1

    diff_iner = clus_ex2.get_moments_of_inertia() - clus_ex1.get_moments_of_inertia()
    
    print(i,shift_move, len(z1), checkSimilar(clus_ex2, clus_ex1), ene2, ene1, diff_iner )
    
    minima_bool_list = []
    for clus in minima_list:      
        minima_bool_list.append(checkSimilar(clus, clus_ex2))
    
    print(minima_bool_list)     
    if any(minima_bool_list):
        pass
    else:
        minima_list.append(clus_ex2)
        min_ene_list.append(ene2)
        traj_atom_list.append(clus_ex2)
            
    print(len(minima_list) )   
    print(len(min_ene_list))
    print(len(traj_atom_list))
    print(min_ene_list)
    #print(minima_list)
    print(reward_list)
    write("final_minima.traj", traj_atom_list)
    print('\n')
    return 

In [38]:
for i in range(20):
    print('iteration', i)
    test_new()
#rint(reward)

iteration 0
2 [ 0 -1  0] 1 False 8.139723889922672 8.330957926629369 [ -166.08105724  -210.55962522 -1502.60726283]
[False]
2
2
2
[8.330957926629369, 8.139723889922672]
[1000.0]


iteration 1
3 [ 0  0 -1] 0 True 8.331053730105353 8.330957926629369 [-8.41499515  7.58106482  9.40527371]
[True, False]
2
2
2
[8.330957926629369, 8.139723889922672]
[1000.0]


iteration 2
3 [0 1 0] 1 False 8.936179934067253 8.330957926629369 [-303.66713458 1848.7041275   757.52623272]
[False, False]
3
3
3
[8.330957926629369, 8.139723889922672, 8.936179934067253]
[1000.0, 1000.0]


iteration 3
11 [ 0 -1  0] 1 False 8.28802106963185 8.330957926629369 [-646.52931623  602.15391009 -218.25271401]
[False, False, False]
4
4
4
[8.330957926629369, 8.139723889922672, 8.936179934067253, 8.28802106963185]
[1000.0, 1000.0, 1000.0]


iteration 4
10 [0 0 1] 0 False 8.139370798142734 8.330957926629369 [ -165.44610463  -211.30908938 -1514.57074987]
[False, True, False, False]
4
4
4
[8.330957926629369, 8.139723889922672, 8.936

In [39]:
min_ene_list = []
minima_list = []
traj_atom_list = []
reward_list  = []
def test_new2():
    ele_Names = ['Ni']
    ele_Nums = [13]
    clus_example = ClusEnv(ele_Names, ele_Nums)
    clus_ex1,d = clus_example.gen_clus()
    
    ene1 = clus_ex1.get_potential_energy()
    if clus_ex1 not in minima_list:
        minima_list.append(clus_ex1)
        traj_atom_list.append(clus_ex1)
    if ene1 not in min_ene_list:
        min_ene_list.append(ene1)
    
    
    random.seed()
    i = random.randint(0,len(clus_ex1)-1)
    shift_move = random.choice(DIRECTION)
    #print(i,shift_move)

    clus_ex2 = clus_ex1.copy()
    clus_ex2[i].position =  clus_ex1.get_positions()[i] + shift_move * 2.5
    
    dist_after = clus_ex2.get_all_distances()[i]
    z1 = [ dist for k, dist in enumerate(dist_after) if  k != i and dist < d ]
    
    #reward = 0.0
    if len(z1) > 0:
        print('Overlapped atoms')
        reward = -100.0
        reward_list.append(reward)
    
    else:   
        clus_ex2.calc = EMT()
        dyn= BFGS(clus_ex2, logfile='test.log')
        dyn.run(fmax=0.02, steps=1000)

    
        ene2 = clus_ex2.get_potential_energy()
        diff_ene = ene2 - ene1
        
        #reward = 100 * np.exp(10*diff_ene)
        #reward_list.append(reward)

        diff_iner = clus_ex2.get_moments_of_inertia() - clus_ex1.get_moments_of_inertia()
    
        #print(i,shift_move, len(z1), checkSimilar(clus_ex2, clus_ex1), ene2, ene1, diff_iner )
    
        minima_bool_list = []
        for clus in minima_list:      
            minima_bool_list.append(checkSimilar(clus, clus_ex2))
    
        print(minima_bool_list)     
        if any(minima_bool_list):
            reward = 0.0
            reward_list.append(reward)
        else:
            reward = 100 * np.exp((-1)*diff_ene)
            reward_list.append(reward)
            minima_list.append(clus_ex2)
            min_ene_list.append(ene2)
            traj_atom_list.append(clus_ex2)
            
    print(len(minima_list) )   
    print(len(min_ene_list))
    print(len(traj_atom_list))
    print(min_ene_list)
    #print(minima_list)
    print(reward_list)
    print(sum(reward_list))
    write("final_minima.traj", traj_atom_list)
    print('\n')
    return 

In [40]:
for i in range(200):
    print('iteration', i)
    test_new2()

iteration 0
Overlapped atoms
1
1
1
[15.529235000970921]
[-100.0]
-100.0


iteration 1
Overlapped atoms
1
1
1
[15.529235000970921]
[-100.0, -100.0]
-200.0


iteration 2
Overlapped atoms
1
1
1
[15.529235000970921]
[-100.0, -100.0, -100.0]
-300.0


iteration 3
Overlapped atoms
1
1
1
[15.529235000970921]
[-100.0, -100.0, -100.0, -100.0]
-400.0


iteration 4
[True]
1
1
1
[15.529235000970921]
[-100.0, -100.0, -100.0, -100.0, 0.0]
-400.0


iteration 5
[False]
2
2
2
[15.529235000970921, 15.189043390619023]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602]
-259.478318114154


iteration 6
[False, False]
3
3
3
[15.529235000970921, 15.189043390619023, 15.563002180206311]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496]
-162.79866123884904


iteration 7
Overlapped atoms
3
3
3
[15.529235000970921, 15.189043390619023, 15.563002180206311]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0]
-262.79866123884904


iteration 8
[True, Fal

[False, False, False, False, False, False, False, False, False, False]
11
11
11
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902]
187.2186389664538


iteration 28
[False, False, False, False, False, False, False, False, True, False, False]
11
11
11
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.

Overlapped atoms
13
13
13
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0]
-266.59143957896003


iteration 40
[False, False, False, False, False, False, False, False, False, False, False, False, False]
14
14
14
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.52830315575

[False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]
15
15
15
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934, 15.136040770115677, 15.571315772911897]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0, 148.1706154435524, -100.0, -100.0, -100.0, 0.0, -100.0, 0.0, 0.0, 95.8792333926965, 0.0]
-422.54159074271115


iteration 50
Overlapped atoms
15
15
15
[15.529235000

Overlapped atoms
16
16
16
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934, 15.136040770115677, 15.571315772911897, 15.858058261534971]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0, 148.1706154435524, -100.0, -100.0, -100.0, 0.0, -100.0, 0.0, 0.0, 95.8792333926965, 0.0, -100.0, 0.0, -100.0, -100.0, -100.0, 0.0, 71.97702172882853, 0.0, -100.0]
-850.5645690138826


iteration 59
Overlapped atoms
16
16
16
[15.5292

...

[True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
17
17
17
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934, 15.136040770115677, 15.571315772911897, 15.858058261534971, 15.5375803867428]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0, 148.1706154435524, -100.0, -100.0, -100.0, 0.0, -100.0, 0.0, 0.0, 95.8792333926965, 0.0, -100.0, 0.0, -100.0, -

[True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
17
17
17
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934, 15.136040770115677, 15.571315772911897, 15.858058261534971, 15.5375803867428]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0, 148.1706154435524, -100.0, -100.0, -100.0, 0.0, -100.0, 0.0, 0.0, 95.8792333926965, 0.0, -100.0, 0.0, -100.0, -

Overlapped atoms
17
17
17
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934, 15.136040770115677, 15.571315772911897, 15.858058261534971, 15.5375803867428]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0, 148.1706154435524, -100.0, -100.0, -100.0, 0.0, -100.0, 0.0, 0.0, 95.8792333926965, 0.0, -100.0, 0.0, -100.0, -100.0, -100.0, 0.0, 71.97702172882853, 0.0, -100.0, -100.0, -100.0, 0.0, 0.0, -100.0, -100.0, 0.0, -10

[False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False]
17
17
17
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934, 15.136040770115677, 15.571315772911897, 15.858058261534971, 15.5375803867428]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0, 148.1706154435524, -100.0, -100.0, -100.0, 0.0, -100.0, 0.0, 0.0, 95.8792333926965, 0.0, -100.0, 0.0, -100.0, -

Overlapped atoms
17
17
17
[15.529235000970921, 15.189043390619023, 15.563002180206311, 14.925752927613003, 15.078344224381725, 15.198218247430674, 15.559136031537196, 15.528303155758055, 15.506580546819507, 15.523809508048675, 15.872037565738852, 15.83444881956732, 15.850910444498934, 15.136040770115677, 15.571315772911897, 15.858058261534971, 15.5375803867428]
[-100.0, -100.0, -100.0, -100.0, 0.0, 140.52168188584602, 96.67965687530496, -100.0, 0.0, 0.0, 182.84746109977746, 0.0, -100.0, -100.0, -100.0, 156.9709823668822, 139.23831194594186, -100.0, -100.0, 0.0, 97.05415827463129, 100.09322795155072, 0.0, 0.0, 102.29130151254462, 100.54402375625573, 0.0, 70.97783329771902, 0.0, 0.0, 0.0, 0.0, -100.0, 73.69657805551888, -100.0, 72.49334339906727, -100.0, -100.0, -100.0, -100.0, 148.1706154435524, -100.0, -100.0, -100.0, 0.0, -100.0, 0.0, 0.0, 95.8792333926965, 0.0, -100.0, 0.0, -100.0, -100.0, -100.0, 0.0, 71.97702172882853, 0.0, -100.0, -100.0, -100.0, 0.0, 0.0, -100.0, -100.0, 0.0, -10

In [41]:
min_ene_list = []
def test_new4():
    ele_Names = ['Pt']
    ele_Nums = [10]
    clus_example = ClusEnv(ele_Names, ele_Nums)
    clus_ex1,d = clus_example.gen_clus()
    
    random.seed()
    i = random.randint(0,len(clus_ex1)-1)
    shift_move = random.choice(DIRECTION)
    #print(i,shift_move)
    ene1 = clus_ex1.get_potential_energy()
    iner1 = clus_ex1.get_moments_of_inertia()
    
    pos_initial = clus_ex1.get_positions()
    #print(pos_initial)
    clus_ex1_copy = clus_ex1.copy()
    
    #clus_ex1 = clus_ex1.copy()
    clus_ex1[i].position =  clus_ex1.get_positions()[i] + shift_move * 1.5
    
    pos_final = clus_ex1.get_positions()
    print(pos_final)
    
    dist_after = clus_ex1.get_all_distances()[i]
    z1 = [ dist for k, dist in enumerate(dist_after) if  k != i and dist < d ]
    
    #clus_ex2.calc = EMT()
    dyn= BFGS(clus_ex1, logfile='test.log')
    dyn.run(fmax=0.02, steps=1000)

    #ene1 = clus_ex1.get_potential_energy()
    ene2 = clus_ex1.get_potential_energy()
    iner2 = clus_ex1.get_moments_of_inertia()
    
    diff_ene = ene2 - ene1
    diff_iner = iner2 -iner1
    
    #print(pos_final - pos_initial)
    #if  checkSimilar(clus_ex1, clus_ex1_copy):
    #xx checkSimilar(clus_ex2, clus_ex1)
    print(i,shift_move, len(z1), checkSimilar(clus_ex1, clus_ex1_copy), ene2, ene1, diff_iner )
    min_ene_list.append(ene2)
    print('\n')
    return 

In [42]:
for i in range(300):
    print('iteration', i)
    test_new4()
#rint(reward)

iteration 0
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 11.83593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
3 [0 0 1] 1 False 7.90666841162599 8.20463596250642 [-661.67132079  595.22492939 -536.88064333]


iteration 1
[[ 8.68697916 13.65210458  9.40012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
0 [ 0  0 -1] 0 True 8.204617910709754 8.20463596250642 [ 0.44454083 -4.06306738 -2.35857444]


i

5 [0 1 0] 0 True 8.20457473597614 8.20463596250642 [-0.21014587  0.2676607   4.11333201]


iteration 17
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 14.44272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
8 [0 1 0] 0 True 8.204616032167817 8.20463596250642 [ 0.06101324 -2.16507572 -9.22855862]


iteration 18
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259  9.47122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
1 [ 0  0 

3 [0 0 1] 1 False 7.90666841162599 8.20463596250642 [-661.67132079  595.22492939 -536.88064333]


iteration 34
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   11.93294418]]
9 [0 0 1] 0 True 8.204691066035183 8.20463596250642 [-5.31160527  7.6838883  11.56230893]


iteration 35
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619  8.83593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
3 

7 [-1  0  0] 0 True 8.2045889772648 8.20463596250642 [-1.09889051 -4.09485712  0.9367713 ]


iteration 51
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619  8.83593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
3 [ 0  0 -1] 0 True 8.204739954667405 8.20463596250642 [-1.06726183 -0.39681046 -5.77084781]


iteration 52
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [12.32685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
7 [1

0 [ 0 -1  0] 0 True 8.204598344181482 8.20463596250642 [ 0.5513025   5.2006589  -2.35844648]


iteration 68
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946    8.93294418]]
9 [ 0  0 -1] 0 True 8.20466828944923 8.20463596250642 [-1.09597072  2.44157444 -3.45107523]


iteration 69
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [11.37679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
4 [

...

3 [ 0 -1  0] 0 True 8.204606368603777 8.20463596250642 [-0.97379363  0.04335665 -6.98837041]


iteration 221
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 8.37679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
4 [-1  0  0] 0 True 8.20474667697045 8.20463596250642 [-4.0914984   7.34502061 -4.31398978]


iteration 222
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 13.8166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
4

7 [0 0 1] 1 False 8.072996355229078 8.20463596250642 [-309.21417711   45.60468448 -557.39454739]


iteration 238
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 15.09421991]
 [ 9.41958568 16.008946   10.43294418]]
8 [0 0 1] 0 True 8.20470665987246 8.20463596250642 [-0.34185559  3.06999712  2.74278712]


iteration 239
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [10.78977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]


6 [ 0 -1  0] 0 True 8.20456276858284 8.20463596250642 [0.90086342 1.02013422 1.4398842 ]


iteration 255
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 11.44272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
8 [ 0 -1  0] 0 True 8.204735267576712 8.20463596250642 [-1.09251971  1.55371796  1.17400332]


iteration 256
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946    8.93294418]]
9 [ 

[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 11.83593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
3 [0 0 1] 1 False 7.90666841162599 8.20463596250642 [-661.67132079  595.22492939 -536.88064333]


iteration 273
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 14.43605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
1 [0 1 0] 0 True 8.204705360947361 8.20463596250642 [-1.60517485  4.20239873 -7.96477839]


iteration 274


3 [0 1 0] 1 True 8.204596177277885 8.20463596250642 [ 1.93162961 -0.97824262 -0.71023133]


iteration 289
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 12.93605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 17.508946   10.43294418]]
9 [0 1 0] 0 True 8.204648621647985 8.20463596250642 [-2.51803418  5.3488616   1.13047442]


iteration 290
[[ 8.68697916 13.65210458 10.90012603]
 [12.61954163 14.43605259 10.97122344]
 [10.85901017 14.65743655 12.09049765]
 [10.36990521 11.74380619 10.33593072]
 [ 9.87679369 12.3166127  12.81708657]
 [11.84946278 10.8013967  12.22666965]
 [13.392327   14.86873812 12.42285651]
 [10.82685068 14.16834551  9.47404197]
 [12.28977103 12.94272771 13.59421991]
 [ 9.41958568 16.008946   10.43294418]]
1 [0 1

In [ ]:
history['timesteps'] = [0]
history['timesteps'] = history['timesteps'] + [history['timesteps'][-1] + 1]

In [ ]:
print(history)

In [ ]:
print(history)

In [ ]:
history['timesteps'] = history['timesteps'] + [history['timesteps'][-1] + 1]
print(history)

In [ ]:
[history['timesteps'][-1] + 1]

In [ ]:
history['timesteps']

In [ ]:
[0, 1, 2] + [3]

In [ ]:
minima = {}
minima['energies'] = [0.3, -0.32, -1.32, 0.4, 2.0, -3.24, -0.11, 0.001, -0.001, -0.5, 0.43]

int(np.argmax(minima['energies']))

In [ ]:
[np.array([0,0,0]).tolist()]

In [ ]:
clus_ex1.get_positions()

In [ ]:
clus_ex1.get_positions().tolist()

In [ ]:
observation = {'energy':np.array(0.23).reshape(1,)}
observation

In [ ]:
clus_ex1.positions

In [ ]:
clus_ex1.get_positions()

In [ ]:
clus_ex1.get_positions() - clus_ex1.positions

In [ ]:
elenums = [4,5,6]
eleradii = [1.26, 1.65, 1.51]
avg = sum(eleradii)/len(elenums)
avg

In [ ]:
l2[1] = 100
l2

In [ ]:
l1

In [ ]:
l1[1] = 10000

In [ ]:
l1

In [ ]:
l2

In [ ]:
l2[2][1] = 500

In [ ]:
l2

In [ ]:
l1

In [ ]:
self.history['timesteps'] = self.history['timesteps'] + [self.history['timesteps'][-1] + 1]
        self.history['energies'] = self.history['energies'] + [self.relative_energy]
        self.history['positions'] = self.history['positions'] + [self.atoms.get_positions(wrap=False).tolist()]
        self.history['scaled_positions'] = self.history['scaled_positions'] + [self.atoms.get_scaled_positions(wrap=False).tolist()]
        if self.observation_fingerprints:
                 self.history['fingerprints'] = self.history['fingerprints'] + [self.fps.tolist()]
                 self.history['initial_fps'] = self.history['initial_fps'] + [self.episode_initial_fps.tolist()]


In [ ]:
self.minima['minima'].append(self.atoms.copy())
                        self.minima['energies'].append(self._get_relative_energy())
                        self.minima['timesteps'].append(self.history['timesteps'][-1] + 1)
                        self.minima['positions'].append(self.atoms.positions.copy())


In [ ]:
self.all_minima['minima'].append(self.atoms.copy())
                self.all_minima['energies'].append(self._get_relative_energy())
                self.all_minima['timesteps'].append(self.history['timesteps'][-1] + 1)
                self.all_minima['positions'].append(self.atoms.positions.copy())


In [ ]:
string = 'I love India'
str_list = string.split()
print(str_list)
for letter in str_list:
    print("".join(str_list))
    a = "".join(letter)
    print(a)
bb =[]
for letter in string:
    bb.append(letter)
print("bb", bb)
cc = "#".join(bb)
print('cc', cc)

In [ ]:
"aaa".join("IamRajesh")

In [ ]:
eleNames = ['Ni', 'Cu']
eleNums = [6, 5]
clus_example = ClusEnv(ele_Names, ele_Nums)
eleradii = [covalent_radii[atomic_numbers[ele]] for ele in eleNames]
eleradii

In [ ]:
relative_energy = +0.3

1000 * np.exp((-10) * relative_energy)

In [ ]:
relative_energy = +1
1000 * np.exp((-1) * relative_energy)

In [ ]:
relative_energy = -0.003
1000 * np.exp( -1/relative_energy)

In [ ]:
relative_energy = -0.005
100/np.exp( relative_energy)

In [ ]:
relative_energy = -0.005
100 * np.exp( -relative_energy)

In [ ]:
x = np.random.randint(low=0, high=50, size=100)
x

In [ ]:
np.arange(0, len(x)+1, 5)

In [ ]:
np.linspace(0, 0.02, num=5)